In [1]:
import os
# Change native directory to root
os.chdir(os.path.dirname(os.getcwd()))

In [27]:
import glob
import pandas as pd
import numpy as np
import random
from tsfeatures import tsfeatures
import matplotlib.pyplot as plt
import plotly.express as px

# K-mean clustering libraries
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import calinski_harabasz_score

In [42]:
# Read the features from the csv file
features = pd.read_csv('data/features.csv', index_col=0)
features

,DN_HistogramMode_5,DN_HistogramMode_10,CO_f1ecac,CO_FirstMin_ac,CO_HistogramAMI_even_2_5,CO_trev_1_num,MD_hrv_classic_pnn40,SB_BinaryStats_mean_longstretch1,SB_TransitionMatrix_3ac_sumdiagcov,PD_PeriodicityWang_th0_01,...,FC_LocalSimple_mean1_tauresrat,DN_OutlierInclude_p_001_mdrmd,DN_OutlierInclude_n_001_mdrmd,SP_Summaries_welch_rect_area_5_1,SB_BinaryStats_diff_longstretch0,SB_MotifThree_quantile_hh,SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1,SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1,SP_Summaries_welch_rect_centroid,FC_LocalSimple_mean3_stderr
411,-0.229518,-0.498146,23,49,0.714297,0.002028,0.518624,60.0,0.002018,95,...,0.727273,-0.537143,0.008972,0.990633,27.0,1.467418,0.26,0.20,0.065434,0.198818
1208,-0.421911,-0.793071,20,49,0.434424,0.006392,0.813468,158.0,0.001414,95,...,0.033333,-0.246978,-0.023704,0.953550,15.0,1.541064,0.24,0.12,0.065434,0.350191
588,-0.350266,-0.787705,21,51,0.351503,-0.025644,0.762562,86.0,0.000186,94,...,0.025000,-0.253764,-0.048315,0.963800,15.0,1.698899,0.22,0.20,0.059106,0.331352
1116,1.185587,0.372905,30,47,0.072859,0.012362,0.405199,1484.0,0.003637,76,...,0.000994,-0.480317,0.063013,0.959287,11.0,1.779463,0.24,0.22,0.025119,0.326308
279,-0.494994,-0.873380,41,44,0.277079,0.106722,0.671359,1072.0,0.005102,95,...,0.000142,-0.357037,0.055272,0.856057,12.0,1.730995,0.20,0.82,0.023633,0.533461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1660,0.121500,-0.470584,18,37,0.206636,-0.014362,0.785829,880.0,0.001562,45,...,0.006231,-0.421728,-0.001283,0.960805,13.0,1.546438,0.22,0.12,0.065434,0.350073
642,-0.248563,-0.539083,21,33,0.443825,-0.004426,0.660780,173.0,0.016389,36,...,0.002387,-0.283987,0.088076,0.968246,12.0,1.462734,0.18,0.12,0.065194,0.337450
597,0.361296,-0.010152,27,2,0.353650,-0.002056,0.885216,851.0,0.001394,3,...,0.007143,-0.447793,0.157913,0.892682,11.0,1.759513,0.24,0.12,0.046786,0.442868
235,-0.282135,-0.673022,17,50,0.357122,-0.000171,0.645478,127.0,0.003658,95,...,0.028571,0.012004,0.017578,0.939083,12.0,1.679973,0.26,0.22,0.065434,0.411346


In [43]:
# read metadata csv
metadata = pd.read_csv('data/EANLIJST_METADATA.csv', index_col=0, sep   = ';')
# ADD the functietype column to the features
features['function'] = metadata['Patrimonium Functietype']
# read more metrics from csv
metrics = pd.read_csv('data/ts_metrics.csv', usecols = ['ID', 'mean', 'std'], index_col='ID')
# add the metrics to the features
features = features.join(metrics)
features

,DN_HistogramMode_5,DN_HistogramMode_10,CO_f1ecac,CO_FirstMin_ac,CO_HistogramAMI_even_2_5,CO_trev_1_num,MD_hrv_classic_pnn40,SB_BinaryStats_mean_longstretch1,SB_TransitionMatrix_3ac_sumdiagcov,PD_PeriodicityWang_th0_01,...,SP_Summaries_welch_rect_area_5_1,SB_BinaryStats_diff_longstretch0,SB_MotifThree_quantile_hh,SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1,SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1,SP_Summaries_welch_rect_centroid,FC_LocalSimple_mean3_stderr,function,mean,std
411,-0.229518,-0.498146,23,49,0.714297,0.002028,0.518624,60.0,0.002018,95,...,0.990633,27.0,1.467418,0.26,0.20,0.065434,0.198818,NaN,398.241228,216.284313
1208,-0.421911,-0.793071,20,49,0.434424,0.006392,0.813468,158.0,0.001414,95,...,0.953550,15.0,1.541064,0.24,0.12,0.065434,0.350191,Stadhuis/Gemeentehuis,9.313439,3.516002
588,-0.350266,-0.787705,21,51,0.351503,-0.025644,0.762562,86.0,0.000186,94,...,0.963800,15.0,1.698899,0.22,0.20,0.059106,0.331352,Academie,4.929265,4.023414
1116,1.185587,0.372905,30,47,0.072859,0.012362,0.405199,1484.0,0.003637,76,...,0.959287,11.0,1.779463,0.24,0.22,0.025119,0.326308,Cultureel centrum,0.675075,1.535042
279,-0.494994,-0.873380,41,44,0.277079,0.106722,0.671359,1072.0,0.005102,95,...,0.856057,12.0,1.730995,0.20,0.82,0.023633,0.533461,NaN,6.109993,4.202058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1660,0.121500,-0.470584,18,37,0.206636,-0.014362,0.785829,880.0,0.001562,45,...,0.960805,13.0,1.546438,0.22,0.12,0.065434,0.350073,Cultureel centrum,7.875420,4.729058
642,-0.248563,-0.539083,21,33,0.443825,-0.004426,0.660780,173.0,0.016389,36,...,0.968246,12.0,1.462734,0.18,0.12,0.065194,0.337450,NaN,15.234523,18.363643
597,0.361296,-0.010152,27,2,0.353650,-0.002056,0.885216,851.0,0.001394,3,...,0.892682,11.0,1.759513,0.24,0.12,0.046786,0.442868,NaN,87.934478,18.172144
235,-0.282135,-0.673022,17,50,0.357122,-0.000171,0.645478,127.0,0.003658,95,...,0.939083,12.0,1.679973,0.26,0.22,0.065434,0.411346,Werkplaats,5.620527,4.201984


In [44]:
# show variance of each feature in the features dataframe
features.var().sort_values(ascending=False)


SB_BinaryStats_mean_longstretch1               9.720957e+06
PD_PeriodicityWang_th0_01                      4.807695e+06
CO_f1ecac                                      2.868387e+06
CO_FirstMin_ac                                 1.815816e+05
mean                                           1.070565e+03
std                                            4.430246e+02
IN_AutoMutualInfoStats_40_gaussian_fmmi        9.204287e+01
SB_BinaryStats_diff_longstretch0               1.618855e+01
DN_HistogramMode_5                             1.619649e+00
CO_trev_1_num                                  7.989520e-01
DN_HistogramMode_10                            5.339645e-01
SB_MotifThree_quantile_hh                      1.333191e-01
DN_OutlierInclude_p_001_mdrmd                  1.146922e-01
MD_hrv_classic_pnn40                           6.420649e-02
CO_HistogramAMI_even_2_5                       5.596611e-02
SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1         5.139684e-02
DN_OutlierInclude_n_001_mdrmd           

In [45]:
# show nan values and drop rows with nan values
features.isnull().sum()
features.dropna(inplace=True)

In [114]:
# K-mean clustering on the features dataset
kmeans = KMeans(n_clusters=7, random_state=0).fit(features.drop(['function'], axis=1))
clusters = kmeans.labels_
features['cluster'] = clusters

In [139]:
subset = features[['mean', 'std', 'FC_LocalSimple_mean3_stderr']]
# Scale all numerical features with standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit_transform(subset)
# K-mean clustering on the features dataset
kmeans = KMeans(n_clusters=7, random_state=0).fit(subset)
clusters = kmeans.labels_
# Create two axis with PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
components = pca.fit_transform(subset)
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}
scat = features.join(pd.DataFrame(components, index=features.index, columns=list(labels.values())))
scat['ID'] = scat.index
scat['cluster'] = clusters
scat

,DN_HistogramMode_5,DN_HistogramMode_10,CO_f1ecac,CO_FirstMin_ac,CO_HistogramAMI_even_2_5,CO_trev_1_num,MD_hrv_classic_pnn40,SB_BinaryStats_mean_longstretch1,SB_TransitionMatrix_3ac_sumdiagcov,PD_PeriodicityWang_th0_01,...,SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1,SP_Summaries_welch_rect_centroid,FC_LocalSimple_mean3_stderr,function,mean,std,cluster,PC 1 (95.9%),PC 2 (4.1%),ID
1208,-0.421911,-0.793071,20,49,0.434424,0.006392,0.813468,158.0,0.001414,95,...,0.12,0.065434,0.350191,Stadhuis/Gemeentehuis,9.313439,3.516002,6,-8.380178,-3.034623,1208
588,-0.350266,-0.787705,21,51,0.351503,-0.025644,0.762562,86.0,0.000186,94,...,0.20,0.059106,0.331352,Academie,4.929265,4.023414,0,-12.168315,-0.769989,588
1116,1.185587,0.372905,30,47,0.072859,0.012362,0.405199,1484.0,0.003637,76,...,0.22,0.025119,0.326308,Cultureel centrum,0.675075,1.535042,0,-17.069314,-1.289834,1116
144,-0.533987,-0.745984,22,50,0.659145,-0.008065,0.528965,324.0,0.000427,95,...,0.20,0.065434,0.268958,Lagere school,4.773989,4.691106,0,-12.035372,-0.097314,144
510,0.373967,0.001066,7,39,0.170913,0.036794,0.111942,62.0,0.119904,94,...,0.12,0.152487,0.577336,Andere gebouwen,1.146714,3.083926,0,-16.002950,-0.072390,510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,-0.426275,-0.741556,25,49,0.463603,0.001404,0.628855,972.0,0.000450,95,...,0.24,0.063133,0.296042,Ontmoetingscentrum,4.722174,3.758550,0,-12.465938,-0.926227,56
13,-1.221434,-0.888564,21,48,0.670719,0.001741,0.595067,165.0,0.001302,95,...,0.12,0.065434,0.265286,Administratief centrum,10.530915,5.580256,6,-6.421759,-1.652993,13
1660,0.121500,-0.470584,18,37,0.206636,-0.014362,0.785829,880.0,0.001562,45,...,0.12,0.065434,0.350073,Cultureel centrum,7.875420,4.729058,0,-9.192511,-1.337713,1660
235,-0.282135,-0.673022,17,50,0.357122,-0.000171,0.645478,127.0,0.003658,95,...,0.22,0.065434,0.411346,Werkplaats,5.620527,4.201984,0,-11.464819,-0.891550,235


In [140]:
fig = px.scatter(scat, x=list(labels.values())[0], y=list(labels.values())[1], color='cluster', hover_name='function', hover_data=['ID', 'mean', 'std'])
fig.update_traces(mode="markers")
fig.show()

In [71]:
scaler = StandardScaler()
df=scaler.fit_transform(df)

kmeans_kwargs = {"init": "random","n_init": 20,"max_iter": 1000,"random_state": 1984}
cut_off=0.5
maxvars=3
kmin=2
kmax=8

cols=list(df.columns)
results_for_each_k=[]
vars_for_each_k={}

for k in range(kmin,kmax+1):
    selected_variables=[]
    while(len(selected_variables)<maxvars):
        results=[]
        for col in cols:
            scols=[]
            scols.extend(selected_variables)
            scols.append(col) 
            kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
            kmeans.fit(df[scols])
            results.append(silhouette_score(df[scols], kmeans.predict(df[scols])))
        selected_var=cols[np.argmax(results)]
        selected_variables.append(selected_var)
        cols.remove(selected_var)
    results_for_each_k.append(max(results))
    vars_for_each_k[k]=selected_variables


best_k=np.argmax(results_for_each_k)+kmin
#you can also force a value for k
#best_k=3
selected_variables=vars_for_each_k[best_k]
kmeans = KMeans(n_clusters=best_k, **kmeans_kwargs)
kmeans.fit(df[selected_variables])
clusters=kmeans.predict(df[selected_variables])

%matplotlib inline

fig = plt.figure(figsize=(15,15))
#plt.rcParams['font.size'] = 22
ax = plt.axes(projection="3d")
z_points = df_[selected_variables[0]]
x_points = df_[selected_variables[1]]
y_points = df_[selected_variables[2]]
f1=ax.scatter3D(x_points, y_points, z_points, c=clusters,cmap='Accent',s=300);

ax.set_xlabel(selected_variables[0],fontsize = 20)
ax.set_ylabel(selected_variables[1],fontsize = 20)
ax.set_zlabel(selected_variables[2],fontsize = 20)
ax.legend(clusters)

plt.title('KMeans used on the Europe Datasets',fontsize = 24)
plt.show()

,kwh,ENERGY_TYPE
DATETIME,,
2019-01-01 00:15:00,0.95,A+
2019-01-01 00:30:00,1.00,A+
2019-01-01 00:45:00,1.00,A+
2019-01-01 01:00:00,1.00,A+
2019-01-01 01:15:00,1.00,A+
...,...,...
2021-12-31 23:00:00,1.05,A+
2021-12-31 23:15:00,1.05,A+
2021-12-31 23:30:00,1.10,A+
